In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.filters import gaussian, threshold_otsu, threshold_multiotsu
from skimage.morphology import remove_small_objects, disk, binary_closing
from scipy.ndimage import zoom, binary_dilation
from skimage.measure import label, regionprops
from skimage import io, exposure
from skimage import measure
from skimage import exposure
from czifile import imread
from cellpose import models, plot 
import re
model_cellpose = models.Cellpose(model_type='cyto')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from huggingface_hub import hf_hub_download
import torch

from micro_sam.evaluation.model_comparison import _enhance_image
from micro_sam.automatic_segmentation import get_predictor_and_segmenter, automatic_instance_segmentation
from concurrent.futures import ThreadPoolExecutor, as_completed

c:\Users\sj1205\AppData\Local\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sj1205\AppData\Local\miniconda3\Lib\site-packages\kornia\feature\lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [2]:
MODEL_TYPE   = "vit_b_lm"
CHECKPOINT = hf_hub_download(repo_id="sunny17347/machine_learning_models", filename="SAM_finetuned.pt")
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def display_image(image, path, type):
    """Display the image."""
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"{path} {type}")
    plt.show()

def display_two_images(image1, image2, title1, title2, path):
    """Display two images side-by-side with smaller title font."""
    filename = os.path.basename(path)  # Extract final part of path

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    axes[0].imshow(image1, cmap='gray' if image1.ndim == 2 else None)
    axes[0].set_title(f"{filename} {title1}", fontsize=10)
    axes[0].axis('off')

    axes[1].imshow(image2, cmap='gray' if image2.ndim == 2 else None)
    axes[1].set_title(f"{filename} {title2}", fontsize=10)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

def extract_image_paths(folder):
    """Extract all image file paths from the specified folder."""
    return [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

def read_image(image_path):
    """Read the LSM image from the specified path."""
    return imread(image_path)

def count(mask): 
    """Count the number of unique labels in the mask."""
    return len(np.unique(label(mask))) - 1  # Exclude background label (0)

def extract_channels(image: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Extract green and red channels from the squeezed image (shape: [Z, C, H, W]).""" 
    return image[0], image[1]

def preprocess_green_channel(green_channel):
    """
    Preprocess the green fluorescence channel for better segmentation and inclusion detection.
    - Applies Gaussian blur to reduce noise.
    - Enhances contrast using sigmoid adjustment.
    - Normalizes intensities to [0, 1] for consistent processing.
    """
    confocal_img = gaussian(green_channel, sigma=2)
    confocal_img = exposure.adjust_sigmoid(green_channel, cutoff=0.25)
    confocal_img = normalize_image(confocal_img)
    return confocal_img

    return circular_mask, non_circular_mask
def normalize_image(image):
    """
    Normalize the image to the range [0, 1].
    This is useful for consistent processing across different images.
    """
    return (image - np.min(image)) / (np.max(image) - np.min(image))

def calculate_surface_area(labeled_image: np.ndarray) -> float:
    """Calculate the total surface area for labeled regions."""
    props = regionprops(labeled_image)
    return sum(prop.area for prop in props)

def segment_cells(green_channel):
    """
    Segment whole cells in the green channel using Cellpose.
    - Normalizes image intensity.
    - Suppresses bright spots (e.g., inclusions) to better detect cell boundaries.
    - Applies Gaussian blur for smoother segmentation input.
    - Gradually increases segmentation diameter until at least one cell is detected.
    """
    green_channel = normalize_image(green_channel)
    percentile_99 = np.percentile(green_channel, 99)
    
    # Suppress very bright pixels (inclusions)
    green_channel_remove_inclusions = np.where(green_channel < percentile_99, green_channel, percentile_99)
    green_channel_remove_inclusions = gaussian(green_channel_remove_inclusions, sigma=5)

    # Normalize again after processing
    green_channel_remove_inclusions = normalize_image(green_channel_remove_inclusions)

    # Try different diameters until cells are detected
    diameter = 150
    while diameter < 500:
        masks, flows, styles, diams = model_cellpose.eval(green_channel_remove_inclusions, diameter=diameter, channels=[0, 0])
        labeled_cells = label(masks)
        if np.max(labeled_cells) > 0:
            return labeled_cells
        diameter += 25

    # No cells found
    return None

def filter_objects_by_overlap(source_mask, target_mask):
    """
    Keeps objects in `source_mask` that overlap with `target_mask`.

    Parameters:
        source_mask (ndarray): Binary mask with objects to test (e.g. clustered mitochondria).
        target_mask (ndarray): Binary mask defining inclusion region.

    Returns:
        filtered_mask (ndarray): Binary mask of source objects overlapping with target.
    """
    labeled_source = label(source_mask)
    filtered_mask = np.zeros_like(source_mask, dtype=bool)

    for region in regionprops(labeled_source):
        coords = tuple(region.coords.T)
        if np.any(target_mask[coords]):
            filtered_mask[coords] = True

    return filtered_mask

def find_swiss_cheese_inclusions(inclusion_image, red_channel_thresholded, verbose=False):
    swiss_chess_inclusions = np.zeros_like(inclusion_image)
    regular_inclusions = np.zeros_like(inclusion_image)
    labeled_inclusions = label(inclusion_image)
    for i, inclusion in enumerate(regionprops(labeled_inclusions)):
        mask = labeled_inclusions == inclusion.label
        overlap = mask * red_channel_thresholded
        if np.sum(overlap) > 0:
            swiss_chess_inclusions += mask
        else:
            regular_inclusions += mask
    return swiss_chess_inclusions, regular_inclusions

def extract_touching_objects(mask1, mask2):
    """
    Return a mask containing whole objects in mask1
    that touch mask2.
    """
    touching_objects = np.zeros_like(mask1, dtype=np.uint8)

    labeled_mask = label(mask1)

    for region in regionprops(labeled_mask):
        region_mask = (labeled_mask == region.label)
        overlap = region_mask * mask2

        if np.sum(overlap) > 0:
            touching_objects += region_mask

    return touching_objects

In [4]:
@torch.no_grad()
def analysis(red: np.ndarray, green:np.ndarray, path:str, predictor, segmenter) -> pd.DataFrame:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    data = []
    df_cell_summary = pd.DataFrame()

    print("Starting analysis on ", path)

    inclusion_image_new = automatic_instance_segmentation(
        predictor=predictor,
        segmenter=segmenter,
        input_path=green,
        ndim=2,
    ) > 0


    # Preprocess the green channel
    green_preprocssed = preprocess_green_channel(green)

    
    #display_image(green, path, "Green Channel")

    #return green

    labeled_cells = segment_cells(green_preprocssed)
    #display_image(labeled_cells, path, "Labeled Cells")


    #display_image(inclusion_image, path, "Inclusion Image")

    #display_two_images(green, inclusion_image_new, "Green Channel", "Inclusion Image", path)


    contrast_adjusted_red_normalized = (red - red.min()) / (red.max() - red.min())
    threshold_value_mitochondria = np.mean(contrast_adjusted_red_normalized) + (np.std(contrast_adjusted_red_normalized) * 2) #1.75 works well
    mitochondria_thresholded = contrast_adjusted_red_normalized > threshold_value_mitochondria
    mitochondria_thresholded = remove_small_objects(mitochondria_thresholded, min_size=10)


    #display_two_images(red, mitochondria_thresholded, "Red Channel", "Mitochondria Thresholded", path)


    #display_image(single, path, "Single Mitochondria")
    #display_image(clustered, path, "Clustered Mitochondria")
    cell_count = 1
    for i, cell in enumerate(regionprops(labeled_cells)):
        if cell.area < 100:  # Skip tiny regions likely to be noise
            continue

                # Create a mask for the current cell
        mask = labeled_cells == cell.label

        inclusions_in_cell = extract_touching_objects(inclusion_image_new, mask)
        mitochondria_in_cell = extract_touching_objects(mitochondria_thresholded, mask)
        mitochondria_in_cell = mitochondria_in_cell > 0  # Ensure binary mask

        swiss_cheese_inclusions_in_cell, regular_inclusions_in_cell = find_swiss_cheese_inclusions(inclusions_in_cell, mitochondria_in_cell)
        lds_in_inclusions_in_cell, lds_not_in_inclusions_in_cell = find_swiss_cheese_inclusions(mitochondria_in_cell, inclusions_in_cell)

        #display_two_images(lds_in_inclusions_in_cell, lds_not_in_inclusions_in_cell, "LDS in Inclusions", "LDS not in Inclusions", path)

        lds_in_inclusions_in_cell_original = lds_in_inclusions_in_cell * red
        lds_not_in_inclusions_in_cell_original = lds_not_in_inclusions_in_cell * red
        lds_in_cell_total_original = mitochondria_in_cell * red

        swiss_cheese_inclusions_in_cell_original = swiss_cheese_inclusions_in_cell * green
        regular_inclusions_in_cell_original = regular_inclusions_in_cell * green
        inclusions_total_original = inclusions_in_cell * green
        #print(np.sum(swiss_cheese_inclusions_in_cell_original) / np.sum(swiss_cheese_inclusions_in_cell) if np.sum(swiss_cheese_inclusions_in_cell) > 0 else 0)
        #print(np.sum(regular_inclusions_in_cell_original) / np.sum(regular_inclusions_in_cell) if np.sum(regular_inclusions_in_cell) > 0 else 0)
        data.append({
            'Image Path': path,
            'Cell Number': cell_count,
            'MFI Lipid tox inside inclusion': np.sum(lds_in_inclusions_in_cell_original) / np.sum(lds_in_inclusions_in_cell) if np.sum(lds_in_inclusions_in_cell) > 0 else 0,
            'MFI Lipid tox outside inclusions': np.sum(lds_not_in_inclusions_in_cell_original) / np.sum(lds_not_in_inclusions_in_cell) if np.sum(lds_not_in_inclusions_in_cell) > 0 else 0,
            'MFI of Lipid Tox Total': np.sum(lds_in_cell_total_original) / np.sum(mitochondria_in_cell) if np.sum(mitochondria_in_cell) > 0 else 0,
            'MFI of Swiss Cheese Inclusions': np.sum(swiss_cheese_inclusions_in_cell_original) / np.sum(swiss_cheese_inclusions_in_cell) if np.sum(swiss_cheese_inclusions_in_cell) > 0 else 0,
            'MFI of Regular Inclusions': np.sum(regular_inclusions_in_cell_original) / np.sum(regular_inclusions_in_cell) if np.sum(regular_inclusions_in_cell) > 0 else 0,
            'MFI of Inclusions Total': np.sum(inclusions_total_original) / np.sum(inclusions_in_cell) if np.sum(inclusions_in_cell) > 0 else 0,
        })
        cell_count += 1

    df_cell_summary = pd.DataFrame(data)
    return df_cell_summary

In [5]:
def process_folder(image_folder):
    print(f"→ Starting {image_folder}")
    images_to_analyze = extract_image_paths(image_folder)
    predictor, segmenter = get_predictor_and_segmenter(
        model_type=MODEL_TYPE,
        checkpoint=CHECKPOINT,
        device=DEVICE,
        is_tiled=False,
    )

    df_cell_summary_list = []
    for path in images_to_analyze:
        image = read_image(path)
        image_squeezed = np.squeeze(image) 
        red, green = extract_channels(image_squeezed)
        df_cell_summary = analysis(red, green, path, predictor, segmenter)
        df_cell_summary_list.append(df_cell_summary)

    combined_cell_summary_df = pd.concat(df_cell_summary_list, ignore_index=True)

    # strip “_images” so you get “061725_MFI_SUMMARY.xlsx”
    base = os.path.basename(image_folder.rstrip("/\\"))
    folder_id = base[:-7] if base.endswith("_images") else base
    output_summary_path = os.path.join(os.getcwd(), f"{folder_id}_MFI_SUMMARY.xlsx")

    combined_cell_summary_df.to_excel(output_summary_path, index=False)
    print(f"✓ Finished {image_folder} → {output_summary_path}")
    return output_summary_path



def main():
    folders = [
        '061725_images',
        '62425_images',
        '06052025_images'
    ]

    max_workers = len(folders)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_folder, f): f for f in folders}
        for fut in as_completed(futures):
            folder = futures[fut]
            try:
                fut.result()
            except Exception as e:
                print(f"✗ Error processing {folder}: {e}")

if __name__ == "__main__":
    main()

→ Starting 061725_images
→ Starting 62425_images
→ Starting 06052025_images


c:\Users\sj1205\AppData\Local\miniconda3\Lib\site-packages\torch_em\data\segmentation_dataset.py:274: UserWarning: SegmentationDataset could not be deserialized because of missing full_data/masks, *.png.
The dataset is deserialized in order to allow loading trained models from a checkpoint.
But it cannot be used for further training and will throw an error.
  warnings.warn(msg)
c:\Users\sj1205\AppData\Local\miniconda3\Lib\site-packages\torch_em\data\segmentation_dataset.py:261: UserWarning: SegmentationDataset could not be deserialized because of missing full_data/images_uint8, *.png.
The dataset is deserialized in order to allow loading trained models from a checkpoint.
But it cannot be used for further training and will throw an error.
  warnings.warn(msg)


Starting analysis on  06052025_images\3K_dox_lipidtox_farred_LA1200_01.czi
Starting analysis on  62425_images\3K_dox_lipidtox_farred_LA1200_01.czi


Compute Image Embeddings 2D:   0%|          | 0/1 [00:00<?, ?it/s]

Starting analysis on  061725_images\3K_dox_lipidtox_farred_LA1200_01.czi


...

Starting analysis on  62425_images\3K_dox_lipidtox_farred_UT_07.czi


Initialize instance segmentation with decoder: 100%|██████████| 1/1 [00:02<00:00,  2.16s/it]


Starting analysis on  62425_images\3K_dox_lipidtox_farred_UT_08.czi


Initialize instance segmentation with decoder: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


✓ Finished 62425_images → c:\Users\sj1205\Documents\code\Kara-Lab\synuclein_inclusion_biogenesis_2024\image_analysis\linoleic_acid\62425_MFI_SUMMARY.xlsx
